In [1]:
import pandas as pd
import numpy as np
import recordlinkage 

In [2]:
amazon_train = pd.read_csv("preprocessed_amazon_train.csv", delimiter = ",")
google_train = pd.read_csv("preprocessed_google_train.csv", delimiter = ",")
amazon_test = pd.read_csv("preprocessed_amazon_test.csv", delimiter = ",")
google_test = pd.read_csv("preprocessed_google_test.csv", delimiter = ",")
train_perfect_matching = pd.read_csv("preprocessed_train_perfect_matching.csv", delimiter = ",")
test_perfect_matching = pd.read_csv("preprocessed_test_perfect_matching.csv", delimiter = ",")

In [30]:
sort_indexer_1 = recordlinkage.index.SortedNeighbourhood("name",window=9)
sortNeighbor_index_1 = sort_indexer_1.index(amazon_train,google_train)

In [31]:
sortNeighbor_amazon_id = []
sortNeighbor_google_id = []
# get the amazon id from blocking
for i in range(0,len(sortNeighbor_index_1)):
    sortNeighbor_amazon_id.append(amazon_train["idAmazon"][sortNeighbor_index_1[i][0]])
# get the google id from blocking
for i in range(0,len(sortNeighbor_index_1)):
    sortNeighbor_google_id.append(google_train["idGoogle"][sortNeighbor_index_1[i][1]])  
    
sortNeighbor_potential_matching_1 = pd.DataFrame({"idAmazon":sortNeighbor_amazon_id,"idGoogle":sortNeighbor_google_id})
print(len(sortNeighbor_potential_matching_1))

5845


In [32]:
# Generate the labels
def negatives_generator(perfect_matching,potential_matching):
    # check the quality of blcoking
    auxiliary = pd.merge(perfect_matching,potential_matching, on=["idAmazon","idGoogle"], how="outer", indicator=True)
    print("true positve/recall: "+str(len(*np.where(auxiliary["_merge"]=="both"))))
    print("false positive/- samples: "+str(len(*np.where(auxiliary["_merge"]=="right_only"))))
    print("false negative/+ lost: "+str(len(*np.where(auxiliary["_merge"]=="left_only")))+"\n")
    # labelling
    auxiliary["label"] = np.where(auxiliary["_merge"]=="both",1,0) 
    print("No. of positives: "+str(len(*np.where(auxiliary["label"]==1))))
    print("No. of negatives: "+str(len(*np.where(auxiliary["label"]==0)))+"\n")
    auxiliary = auxiliary[["idAmazon","idGoogle","label"]]
#     auxiliary['similarity'].fillna(1,inplace=True) 
    return auxiliary

In [33]:
# Label
train_index_labels = negatives_generator(train_perfect_matching,sortNeighbor_potential_matching_1)
#[similarity, idAmazon, idGoogle, label]

true positve/recall: 358
false positive/- samples: 5487
false negative/+ lost: 708

No. of positives: 358
No. of negatives: 6195

